# Graph Neural Networks with new tracking dataset

Ming Fong

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import libraries and some constants

import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import tensorflow as tf

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

import uproot3 as ur
# import atlas_mpl_style as ampl
# ampl.use_atlas_style()

path_prefix = '/home/mfong/git/LCStudies/'
plotpath = path_prefix + 'classifier/Plots/'
modelpath = path_prefix + 'classifier/Models/'
# %config InlineBackend.figure_format = 'svg'

# metadata
layers = ["EMB1", "EMB2", "EMB3", "TileBar0", "TileBar1", "TileBar2"]
cell_size_phi = [0.098, 0.0245, 0.0245, 0.1, 0.1, 0.1]
cell_size_eta = [0.0031, 0.025, 0.05, 0.1, 0.1, 0.2]
len_phi = [4, 16, 16, 4, 4, 4]
len_eta = [128, 16, 8, 4, 4, 2]

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# set up GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.test.gpu_device_name())

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=24220)]) #in MB

print("Num GPUs Available: ", len(gpus))
tf.config.list_physical_devices()

/device:GPU:0
Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Jan's notebook

https://github.com/janTOffermann/LCStudies/blob/master/classifier/input-update.ipynb

In [ ]:
# TODO look at new data with positions and make edges relative distances

# pi0
# /clusterfs/ml4hep/mfong/ML4Pions/MLTreeData/user.angerami.mc16_13TeV.900246.PG_singlepi0_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/user.angerami.24559740.OutputStream._000011.root
# pion
# /clusterfs/ml4hep/mfong/ML4Pions/MLTreeData/user.angerami.mc16_13TeV.900247.PG_singlepion_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/user.angerami.24559744.OutputStream._000001.root

MLTreeData_dir = "/clusterfs/ml4hep/mfong/ML4Pions/MLTreeData/"
pi0_dir = MLTreeData_dir + "user.angerami.mc16_13TeV.900246.PG_singlepi0_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/"
pion_dir = MLTreeData_dir + "user.angerami.mc16_13TeV.900247.PG_singlepion_logE0p2to2000.e8312_e7400_s3170_r12383.v01-45-gaa27bcb_OutputStream/"
pi0_root = pi0_dir + "user.angerami.24559740.OutputStream._000011.root"
pion_root = pion_dir + "user.angerami.24559744.OutputStream._000001.root"

pi0_file = ur.open(pi0_root)
pion_file = ur.open(pion_root)

pi0_file["EventTree;1"].show()

pi0_file["EventTree;1"].arrays("trackEta_EMB1", outputtype=pd.DataFrame, entrystop=100)

pi0_file["EventTree;1"].arrays("trackPhi_EMB1", outputtype=pd.DataFrame, entrystop=100)